In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# !pip install requests

In [3]:
import json
import requests
import torch


## **Loading JSONl**

In [4]:
file_path = '/content/drive/My Drive/FYP content/Final thesis/components.jsonl'

In [5]:
def load_from_jsonl(file_path):
    data = []
    with open(file_path, 'r') as f:
        for i, line in enumerate(f, 1):
            try:
                data.append(json.loads(line))
            except json.JSONDecodeError:
                print(f"Error decoding JSON on line {i}: {line}")
                raise  # Optionally re-raise the exception or continue to try to load other lines
    return data

In [6]:
loaded_data = load_from_jsonl(file_path)

In [7]:
def generate_multiple_prompts(json_data):

    #dealing with all the variations

    variant_props = json_data.get('variant_properties', {})
    component_name = variant_props.get('component_name', 'Component')
    style = variant_props.get('style', 'Professional')
    border_radius = str(variant_props.get('borderRadius', 'default border radius'))
    stroke = str(variant_props.get('strokeWeight', 'default stroke'))


    #dealing with multiple variants
    variant_details = variant_props.get('variant_details', {})
    variant_descriptions = [f"{variant_type} of {' and '.join(variants)}" for variant_type, variants in variant_details.items()]
    variant_description_str = ", ".join(variant_descriptions)

    #dealing with strokes
    stroke_description = f"with a stroke of {stroke}" if stroke is not None else ""

    #dealing with effects
    effects = variant_props.get('effects', [])
    effects_descriptions = [effect['type'] for effect in effects if 'type' in effect]
    effects_description_str = " and ".join(effects_descriptions)

    #the prompts
    prompts = [
        f"Create a {component_name} with a style of {style} and {variant_description_str}.",
        f"Generate a {style} {component_name} with border radius {border_radius} and {variant_description_str}.",
        f"Generate a {style} {component_name} with {variant_description_str}.",
        f"Create a {style} {component_name} with a border radius of {border_radius}.",
        f"Generate a {component_name} with {variant_description_str}.",
    ]

    #adding stroke-related prompts
    if stroke:
        prompts.extend([
            f"Generate a {style} {component_name} with a stroke of {stroke} with {variant_description_str}.",
            f"Generate a {style} {component_name} with a stroke weight of {stroke} and border radius of {border_radius}.",
            f"Generate a {component_name} with a stroke of {stroke} with {variant_description_str}.",
            f"Generate a {component_name} with a stroke weight of {stroke} and border radius of {border_radius}."
        ])

    #adding effects-related prompts
    if effects:
        prompts.extend([
            f"Create a {component_name} with {variant_description_str} with {effects_description_str} effects.",
            f"Generate a {style} {component_name} with {effects_description_str} effects and border radius of {border_radius}."
        ])

    #adding prompts for both strokes and effects
    if effects and stroke:
        prompts.extend([
            f"Generate a {component_name} with a stroke of {stroke}, {effects_description_str} effects, and {variant_description_str}.",
            f"Generate a {style} {component_name} with a stroke of {stroke}, and ed{effects_description_str} effects, and {variant_description_str}.",
            f"Generate a {style} {component_name} with a stroke of {stroke} and {effects_description_str} effects."
        ])

    return prompts

In [8]:
def create_prompts_dataset(file_path):
    loaded_data = load_from_jsonl(file_path)
    all_prompts = []

    for json_data in loaded_data:
        prompts = generate_multiple_prompts(json_data)
        all_prompts.extend(prompts)

    return all_prompts

In [9]:
dataset_prompts = create_prompts_dataset(file_path)

In [10]:
for prompt in dataset_prompts[:5]:
    print(prompt)

Create a Button with a style of Professional and State of Default, Size of Small.
Generate a Professional Button with border radius 10.0 and State of Default, Size of Small.
Generate a Professional Button with State of Default, Size of Small.
Create a Professional Button with a border radius of 10.0.
Generate a Button with State of Default, Size of Small.


## **Save file**

In [12]:
def format_for_t5(json_data):
    #to string
    input_str = json.dumps(json_data)
    #generate prompts
    prompts = generate_multiple_prompts(json_data)
    #format each prompt
    t5_formatted_data = [f"generate prompt: {input_str} [SEP] {prompt}" for prompt in prompts]
    return t5_formatted_data

In [13]:
t5_formatted_dataset = []
for json_data in loaded_data:
    t5_formatted_data = format_for_t5(json_data)
    t5_formatted_dataset.extend(t5_formatted_data)

In [14]:
output_file_path = '/content/drive/My Drive/FYP content/Final thesis/t5_formatted_prompts.txt'

In [15]:
with open(output_file_path, 'w') as f:
    for item in t5_formatted_dataset:
        f.write("%s\n" % item)

Reverse

In [17]:
def format_for_reverse_t5(json_data):
    prompts = generate_multiple_prompts(json_data)
    #convert
    output_str = json.dumps(json_data)
    t5_formatted_data = [f"generate json: {prompt} [SEP] {output_str}" for prompt in prompts]
    return t5_formatted_data

In [18]:
reverse_t5_formatted_dataset = []
for json_data in loaded_data:
    reverse_formatted_data = format_for_reverse_t5(json_data)
    reverse_t5_formatted_dataset.extend(reverse_formatted_data)

In [19]:
output_file_path = '/content/drive/My Drive/FYP content/Final thesis/reverse_t5_formatted_json.txt'
with open(output_file_path, 'w') as f:
    for item in reverse_t5_formatted_dataset:
        f.write("%s\n" % item)